<a href="https://colab.research.google.com/github/RajuMopidevi/Machine-Learning-and-Artificial-Intelligence/blob/main/05-01_LexicalProcessing/spell_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
from collections import Counter

In [6]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [7]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/RajuMopidevi/Machine-Learning-and-Artificial-Intelligence/main/05-01_LexicalProcessing/big.txt", "big.txt")

('big.txt', <http.client.HTTPMessage at 0x7f498f6c0df0>)

In [8]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('big.txt').read()))

In [9]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [10]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [11]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [12]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [13]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [14]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [15]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [16]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'monhey', 'monneh', 'lmonney', 'mobney', 'monnee', 'monnney', 'monneyi', 'monaney', 'monpney', 'moanney', 'xonney', 'monneyx', 'mqnney', 'moiney', 'mounney', 'monsney', 'emonney', 'mononey', 'monyey', 'monnety', 'monnsy', 'monnehy', 'uonney', 'monnaey', 'mokney', 'movney', 'monaey', 'moneny', 'munney', 'mocnney', 'moqney', 'myonney', 'monneyc', 'mognney', 'monnedy', 'monbey', 'monned', 'moqnney', 'mojney', 'mponney', 'monnegy', 'monneyk', 'wonney', 'omnney', 'monnqey', 'lonney', 'moinney', 'monnvey', 'mtnney', 'gonney', 'monneey', 'monneya', 'monnuy', 'monneyn', 'moyney', 'monnwey', 'monneyt', 'moncney', 'eonney', 'movnney', 'monnec', 'umonney', 'monnoy', 'monnen', 'monnsey', 'aonney', 'monnea', 'monnesy', 'monneoy', 'monnwy', 'mmnney', 'hmonney', 'monnezy', 'vonney', 'mbonney', 'motney', 'monzey', 'monneyl', 'monmney', 'nmonney', 'monnzey', 'mlnney', 'mojnney', 'mouney', 'monnty', 'monvey', 'monneyd', 'monnvy', 'mofney', 'rmonney', 'montey', 'tonney', 'monkey', 'monneny', 'mknney

In [17]:
print(known(edits_one("monney")))

{'money', 'monkey'}


In [18]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money', 'monkey'}


In [19]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money', 'monkey'}


In [20]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [21]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [22]:
# test spell check
print(spell_check("monney"))

Did you mean money?
